# ReadMe

This Jupyter Notebook is converting Methyldatel Output to NanoDiP bin format.

Please Note: The Jupyter Notebook ist provided as it is.

Please adjust Path definitions in the configuration setting to your needs.

You have to define sample IDs in the sampleIDlist as well as  ExportFileName to your needs

# configuration

In [ ]:
#Bin index
binIndex="/applications/reference_data/betaEPIC450Kmix_bin/index.csv"

#Path to your directory containing the Outputfile of MehtylDackel
MethDpath ="/media/minknow/16TB_3/Twist/2022_10_06_TrimGalore_BitmapperBS_samtools_MD/"

SampleIDlist=["B2021_9857", "B2021_9858", "B2021_11239",  "B2021_23956", "B2021_24089", "B2021_28670", "B2021_22205_2", "LnB_con"]

#Illumina annotation
ilmncgmapfile="/applications/reference_data/microarray/hg19_HumanMethylation450_15017482_v1-2_cgmap.tsv" # Illumina probe names of the 450K 

#path of directory to which bins are to be written
BinDir="/media/minknow/16TB_3/Twist/2022_11_25_code_test"

# read in array annotation into two dictionaries

In [ ]:
# read ilmncgmap into pandas dataframe and convert it to two dictionaries 
import pandas
def arrayAnnotation():
    ilmncgmap=pandas.read_csv(ilmncgmapfile, delimiter='\t', header=None, index_col=0)
    #ilmncgmap
    ilmncgmap.columns = ['chr','strand','pos']
    ilmncgmap["seqpos"]=ilmncgmap["pos"]+1
    ilmncgmap=ilmncgmap.assign(chrompos=ilmncgmap.chr.map(str) + ":" + ilmncgmap.seqpos.map(str))
    ilmncgdict_cg=ilmncgmap['chrompos'].to_dict()
    ilmncgdict={v: k for k, v in ilmncgdict_cg.items()} # reverse dictionary
    ilmncgdict
    del(ilmncgmap)# remove df from memory
    return(ilmncgdict_cg,ilmncgdict)

# read in array index

In [ ]:
import numpy
def ReadInArrayIndex():
    indexFile=open(binIndex, "r") # load CpG site index file (contains index for methylation float binary data)
    indexCol=indexFile.read().split("\n")# returns a list with an empty entry for the last element
    indexCol=list(filter(None, indexCol)) #remove empty list elements ref: https://stackoverflow.com/questions/3845423/remove-empty-strings-from-a-list-of-strings post 21
    indexFile.close()
    return(indexCol)

# parse MethD bedgraph/BitmapperBS

In [ ]:
import pandas
import numpy

def MethDanalysis(s, ilmncgdict):

    MethD_out = MethDpath + s + "_markdup_CpG.bedGraph"# for filename generated in mfcore MD pipline

    print(MethD_out)

    MethD_Dict = dict()

    with open(MethD_out, "r") as MethDFile:
 
        next(MethDFile)#skip header in first line
        for line in MethDFile:
            line = line.strip() #remove tailing wihte spaces         
            words=line.split()  # split line into list using any seperator

            PosString = words[0] + ":" + words[2]
            if PosString in ilmncgdict:# use only those chromosomal positions that are in the ilmncgdict dictionary - i.e. described in the Illumina manifest
                cg_string = ilmncgdict[PosString]
                MethDbeta = int(words[3])/100 #devide the ration from Methyldackel by 100 to work on the same scale as the array and Bismark evaluation
                EventCount = int(words[4]) + int(words[5])#How oftern this site was covered in the Methyldackeloutput file
                MethD_Dict[cg_string] = [MethDbeta , EventCount]


    MethDFile.close()
    
    
    MethD_Dict
        
    MethD_DF=pandas.DataFrame.from_dict(MethD_Dict, orient='index')
    MethD_DF.columns=["MethDbeta", "MethD_EventCount"]
    MethD_DF["lgMD_EventCount"] = numpy.log10(MethD_DF["MethD_EventCount"])
    MethD_DF
    nrowMethD_DF=len(MethD_DF.index)
    return( MethD_Dict, MethD_DF, nrowMethD_DF)

# Creation of bin files

In [ ]:
import pandas
import numpy
def ConvertToBin(SampleID,SampleBetaDF_i):
    numarray=numpy.array(SampleBetaDF_i['Beta'])
    #ExportFileName=SampleID+"_BitmapperBS20220926_all_reads.bin"
    ExportFileName=SampleID+"_nfcore_MD_20220925_all_reads.bin"
    print("ExportFileName:", ExportFileName)
    BinFile=BinDir+"/"+ ExportFileName
    print("BinFile:", BinFile)
    numpy.asarray(numarray, dtype=float).tofile(BinFile)
    

# main() 

In [ ]:
import pandas as pd

def main():
    ilmncgdict_cg, ilmncgdict = arrayAnnotation() #derive array annotation in two dictionaries
    ArrayIndex=ReadInArrayIndex()
    print(len(ArrayIndex))
    
   
    for sampleID in sampleIDlist:
        
        print("sampleID:", sampleID)
        SampleBetaDF=pd.DataFrame()
        SampleBetaDF.index=ArrayIndex
        
        MethD_Dict, MethDdF, nrowMethD_DF=MethDanalysis(sampleID, ilmncgdict)
            
        for cg in ArrayIndex:
            
            #test if all cg defined in the array are described in the sequening experiment
            #assign to cgs not covered in the seuqencing experiment a beta value of 0.51
            if cg in MethD_Dict.keys():

                beta=MethD_Dict[cg][0]
                
            if cg not in MethD_Dict.keys():
                beta=0.49
                
            SampleBetaDF.loc[cg,"Beta"]=beta    
            

        print("SampleBeta:\n", SampleBetaDF)
    
    #convert Beta colum to  bin format
        ConvertToBin(sampleID,SampleBetaDF)
    

    
if __name__ == '__main__':
    main()